In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix , f1_score
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
credit_card = pd.read_csv('creditcard.csv')
X = credit_card.drop(columns='Class', axis=1)
y = credit_card.Class.values
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y)
y_train, y_test = np.array([-1 if y==0 else 1 for y in y_train]), np.array([-1 if y==0 else 1 for y in y_test])
y_train, y_test = y_train.reshape(-1,1), y_test.reshape(-1,1)
y_train, y_test = y_train.astype(np.float32), y_test.astype(np.float32)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train).astype(np.float32)
X_test = scaler.transform(X_test).astype(np.float32)

In [4]:
class svm():
    def __init__(self):
        super(svm, self).__init__()
    
    def predict(self, x, logits=False):
        if logits:
            return tf.subtract(tf.matmul(x, self.w), self.b)
        else:
            return tf.sign(tf.subtract(tf.matmul(x, self.w), self.b))
    
    def loss(self, pred, true):
        classification_term = tf.reduce_mean(tf.maximum(0., tf.subtract(1., tf.multiply(pred, true))))
        loss = tf.add(classification_term, tf.multiply(self.alpha, tf.reduce_sum(tf.square(self.w))))
        return loss
    def accuracy(self, pred, true):
        check_prediction = tf.equal(tf.sign(pred), true)
        accuracy_op = tf.reduce_mean(tf.cast(check_prediction, tf.float32)) * 100
        return accuracy_op
    
    def fit(self, x, y, n_epochs, batch_size, validation_data=None, alpha=0.1, learning_rate=0.01):
        print(f"epochs={n_epochs}, batch size={batch_size}, learning rate={learning_rate}, alpha={alpha}")
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)
        self.alpha = tf.constant([alpha])
        self.w = tf.Variable(tf.random.truncated_normal([x.shape[1], 1]), name="weight", trainable=True)
        self.b = tf.Variable(tf.zeros([1]), name='bias', trainable=True)     

        train_data=tf.data.Dataset.from_tensor_slices((x, y))
        train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)
        self.train_loss = []
        self.train_accuracy= []
        if validation_data:
            self.valid_accuracy = []
            self.valid_loss = []
        for step, (batch_x, batch_y) in enumerate(train_data.take(n_epochs), 1):
            # Run the optimization to update W and b values.
            with tf.GradientTape() as g:
                pred = tf.subtract(tf.matmul(batch_x, self.w), self.b)
                loss = self.loss(pred, batch_y)
                self.train_loss.append(loss)
                self.train_accuracy.append(self.accuracy(pred, batch_y))
            # Compute gradients.
            gradients = g.gradient(loss, [self.w, self.b])
            self.optimizer.apply_gradients(zip(gradients, [self.w, self.b]))
            if validation_data:
                x_val, y_val = validation_data
                pred_val = self.predict(x_val)
                self.valid_loss.append(self.loss(pred_val, y_val))
                self.valid_accuracy.append(self.accuracy(pred_val, y_val))
                
                if step % 50 == 0:
                    print("Iteration: %i, loss: %.2f, accuracy: %.2f, loss_val: %.2f, accuracy_val: %.2f" % (step, 
                    self.train_loss[step-1], self.train_accuracy[step-1], self.valid_loss[step-1], self.valid_accuracy[step-1]))
            else:
                if step % 50 == 0:
                    print("Iteration: %i, loss: %.4f, accuracy: %.2f" % (step, self.train_loss[step-1], self.train_accuracy[step-1]))

            


In [14]:
model = svm()
model.fit(X_train, y_train, 10000, 500, learning_rate=0.001, alpha=0.003, validation_data=(X_test, y_test))

epochs=10000, batch size=500, learning rate=0.001, alpha=0.003
Iteration: 50, loss: 2.40, accuracy: 49.20, loss_val: 1.06, accuracy_val: 50.75
Iteration: 100, loss: 2.29, accuracy: 48.60, loss_val: 1.04, accuracy_val: 51.44
Iteration: 150, loss: 1.79, accuracy: 55.00, loss_val: 1.02, accuracy_val: 52.14
Iteration: 200, loss: 2.06, accuracy: 52.00, loss_val: 1.00, accuracy_val: 52.80
Iteration: 250, loss: 1.71, accuracy: 53.20, loss_val: 0.98, accuracy_val: 53.69
Iteration: 300, loss: 1.62, accuracy: 57.20, loss_val: 0.96, accuracy_val: 54.70
Iteration: 350, loss: 1.53, accuracy: 58.80, loss_val: 0.93, accuracy_val: 55.69
Iteration: 400, loss: 1.45, accuracy: 56.60, loss_val: 0.90, accuracy_val: 56.89
Iteration: 450, loss: 1.21, accuracy: 64.60, loss_val: 0.87, accuracy_val: 58.26
Iteration: 500, loss: 1.35, accuracy: 59.60, loss_val: 0.84, accuracy_val: 59.67
Iteration: 550, loss: 1.29, accuracy: 60.60, loss_val: 0.81, accuracy_val: 61.26
Iteration: 600, loss: 1.37, accuracy: 60.80, lo

Iteration: 5000, loss: 0.00, accuracy: 100.00, loss_val: 0.00, accuracy_val: 99.94
Iteration: 5050, loss: 0.02, accuracy: 99.60, loss_val: 0.00, accuracy_val: 99.94
Iteration: 5100, loss: 0.00, accuracy: 100.00, loss_val: 0.00, accuracy_val: 99.94
Iteration: 5150, loss: 0.00, accuracy: 100.00, loss_val: 0.00, accuracy_val: 99.94
Iteration: 5200, loss: 0.00, accuracy: 100.00, loss_val: 0.00, accuracy_val: 99.94
Iteration: 5250, loss: 0.00, accuracy: 99.80, loss_val: 0.00, accuracy_val: 99.94
Iteration: 5300, loss: 0.00, accuracy: 100.00, loss_val: 0.00, accuracy_val: 99.94
Iteration: 5350, loss: 0.00, accuracy: 100.00, loss_val: 0.00, accuracy_val: 99.94
Iteration: 5400, loss: 0.00, accuracy: 99.80, loss_val: 0.00, accuracy_val: 99.94
Iteration: 5450, loss: 0.00, accuracy: 100.00, loss_val: 0.00, accuracy_val: 99.94
Iteration: 5500, loss: 0.01, accuracy: 99.80, loss_val: 0.00, accuracy_val: 99.94
Iteration: 5550, loss: 0.00, accuracy: 100.00, loss_val: 0.00, accuracy_val: 99.94
Iteratio

Iteration: 10000, loss: 0.00, accuracy: 100.00, loss_val: 0.00, accuracy_val: 99.94


In [15]:
y_train_hat = model.predict(X_train)
# y_train_hat_probs = lr.predict_proba(X_train)[:,1]
train_accuracy = accuracy_score(y_train, y_train_hat)*100
# train_auc_roc = roc_auc_score(y_train, y_train_hat_probs)*100
print('Confusion matrix:\n', confusion_matrix(y_train, y_train_hat))
print('Training accuracy: %.4f %%' % train_accuracy)
# print('Training AUC: %.4f %%' % train_auc_roc)

Confusion matrix:
 [[213191     35]
 [    90    289]]
Training accuracy: 99.9415 %


In [16]:
y_test_hat = model.predict(X_test)
# y_test_hat_probs = lr.predict_proba(X_test)[:,1]
test_accuracy = accuracy_score(y_test, y_test_hat)*100
# test_auc_roc = roc_auc_score(y_test, y_test_hat_probs)*100
print('Confusion matrix:\n', confusion_matrix(y_test, y_test_hat))
print('Test accuracy: %.4f %%' % test_accuracy)
# print('Test AUC: %.4f %%' % test_auc_roc)

Confusion matrix:
 [[71072    17]
 [   28    85]]
Test accuracy: 99.9368 %


In [17]:
print(classification_report(y_test, y_test_hat, digits=6))

              precision    recall  f1-score   support

        -1.0   0.999606  0.999761  0.999684     71089
         1.0   0.833333  0.752212  0.790698       113

    accuracy                       0.999368     71202
   macro avg   0.916470  0.875987  0.895191     71202
weighted avg   0.999342  0.999368  0.999352     71202



In [ ]:
plt.figure(figsize=(15,6))
plt.plot(lr.train_accuracy, label='train_accuracy')
plt.plot(lr.valid_accuracy, label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(lr.train_loss, label='train_loss')
plt.plot(lr.valid_loss, label='val_loss')
plt.legend()
plt.show()